In [1]:
import vectorbt as vbt
import numpy as np
import pandas as pd
import yfinance as yf
from numba import njit
import ta

In [2]:
# Data
symbol = 'SPXL'  # Using QQQ as an example

strat_name = 'dual_ema_crossover'

freq='1D'

price = vbt.YFData.download(
    symbol,
    start='1/1/2023',
    interval='1d'
).get('Close')

fast_level = np.arange(9,27)
slow_level = np.arange(26,60)
EMA = vbt.IndicatorFactory.from_ta('EMAIndicator')
# ema13 = EMA.run(price, 5)
# ema48 = EMA.run(price, 10)

windows = np.arange(2, 101)
ema13, ema48 = EMA.run_combs(price, window=windows, r=2, short_names=['ema_fast', 'ema_slow'])

entries = ema13.ema_indicator_crossed_above(ema48)
exits = ema48.ema_indicator_crossed_below(ema48)

@njit
def adjust_sl_func_nb(c):
    current_profit = (c.val_price_now - c.init_price) / c.init_price
    if current_profit >= 0.6:
        return 0.4, True
    if current_profit >= 0.40:
        return 0.05, True
    elif current_profit >= 0.2:
        return 0.01, True
    # elif current_profit >= 0.05:
    #     return 0.01, True
    # else:
    #     return 0.2, False
    return c.curr_stop, c.curr_trail

take_profit_pct = 20  # 10%
stop_loss_pct = 0.1  # -5%

# Backtest with vectorbt, specifying short entries and exits
pf = vbt.Portfolio.from_signals(
    close=price,
    entries=entries,
    exits=exits,
    short_entries=exits,
    short_exits=entries,
    # init_cash=10000,
    # size=1,
    # size_type=vbt.portfolio.enums.SizeType.Percent,
    sl_stop=stop_loss_pct,
    adjust_sl_func_nb=adjust_sl_func_nb,
    tp_stop=take_profit_pct,
    freq=freq
)

In [12]:
pf.total_return()[::-1]

AttributeError: 'Portfolio' object has no attribute 'profit_factor'

In [5]:
xx = pf.get_orders()

# pf.cumulative_returns().vbt.plot().show()
max_pf = pf.total_return().idxmax()


In [14]:
max_pf

(2, 3)

In [23]:
feats = list(pf.total_return().index.names)

In [29]:
dict(m=1)

{'m': 1}

In [30]:
best_feats = dict({f:m for f,m in zip(feats, max_pf)})

In [31]:
best_feats

{'ema_fast_window': 2, 'ema_slow_window': 3}

In [ ]:

opf = pf[max_pf]
opf.plot().show()

# Show performance metrics
print(max_pf)


print(opf.stats())


In [62]:
# Make dataframe to store 👉🏽 `strategy_stats`
base_stats = opf.stats()

In [135]:
base_stats['Features'] = best_feats
base_stats['Ticker'] = symbol

In [152]:
df_store = pd.DataFrame(base_stats).T.reset_index(drop=True).convert_dtypes()

In [153]:
import re

def to_snake_case(df):
    """
    Convert the column names of a DataFrame to lowercase snake_case format,
    removing special characters and newlines before conversion.
    
    Parameters:
    - df: pandas.DataFrame
    
    Returns:
    - DataFrame with column names in snake_case format.
    """
    # First, replace newlines with spaces, then remove special characters except spaces
    df.columns = [col.replace('\n', '') for col in df.columns]
    df.columns = [col.replace('[%]', '') for col in df.columns]
    df.columns = [col.strip() for col in df.columns]
    df.columns = [col.replace(' ', '_') for col in df.columns]
    df.columns = [col.lower() for col in df.columns]
    
    return df

In [154]:
df_store

,Start,End,Period,Start Value,End Value,Total Return [%],Benchmark Return [%],Max Gross Exposure [%],Total Fees Paid,Max Drawdown [%],...,Avg Winning Trade Duration,Avg Losing Trade Duration,Profit Factor,Expectancy,Sharpe Ratio,Calmar Ratio,Omega Ratio,Sortino Ratio,Features,Ticker
0,2023-01-03 05:00:00+00:00,2024-04-10 04:00:00+00:00,319 days,100,173.354377,73.354377,108.29801,100,0,32.791902,...,37 days 14:24:00,30 days 08:00:00,2.845573,10.582717,1.678868,2.673454,1.270981,2.528137,"{'ema_fast_window': 2, 'ema_slow_window': 3}",SPXL


In [155]:
to_snake_case(df_store)

,start,end,period,start_value,end_value,total_return,benchmark_return,max_gross_exposure,total_fees_paid,max_drawdown,...,avg_winning_trade_duration,avg_losing_trade_duration,profit_factor,expectancy,sharpe_ratio,calmar_ratio,omega_ratio,sortino_ratio,features,ticker
0,2023-01-03 05:00:00+00:00,2024-04-10 04:00:00+00:00,319 days,100,173.354377,73.354377,108.29801,100,0,32.791902,...,37 days 14:24:00,30 days 08:00:00,2.845573,10.582717,1.678868,2.673454,1.270981,2.528137,"{'ema_fast_window': 2, 'ema_slow_window': 3}",SPXL


In [162]:
df_store.to_csv('strategy_stats.csv')

In [163]:
df_store

,start,end,period,start_value,end_value,total_return,benchmark_return,max_gross_exposure,total_fees_paid,max_drawdown,...,avg_winning_trade_duration,avg_losing_trade_duration,profit_factor,expectancy,sharpe_ratio,calmar_ratio,omega_ratio,sortino_ratio,features,ticker
0,2023-01-03 05:00:00+00:00,2024-04-10 04:00:00+00:00,319 days,100,173.354377,73.354377,108.29801,100,0,32.791902,...,37 days 14:24:00,30 days 08:00:00,2.845573,10.582717,1.678868,2.673454,1.270981,2.528137,"{'ema_fast_window': 2, 'ema_slow_window': 3}",SPXL


In [54]:
xx.config['close'][max_pf]

Date
2018-01-02 05:00:00+00:00     43.365036
2018-01-03 05:00:00+00:00     44.132732
2018-01-04 05:00:00+00:00     44.689304
2018-01-05 05:00:00+00:00     45.533764
2018-01-08 05:00:00+00:00     45.812054
                                ...    
2024-03-25 04:00:00+00:00    131.880005
2024-03-26 04:00:00+00:00    130.809998
2024-03-27 04:00:00+00:00    134.080002
2024-03-28 04:00:00+00:00    134.029999
2024-04-01 04:00:00+00:00    133.193405
Name: (5, 10), Length: 1571, dtype: float64

In [73]:
pf_kwargs = dict(size=np.inf, fees=0.001, freq='1D')

fig = pf.total_return().vbt.heatmap(
    x_level='fast_window', y_level='slow_window', symmetric=True,
    trace_kwargs=dict(colorbar=dict(title='Total return', tickformat='%')))
fig.show()

AttributeError: 'numpy.float64' object has no attribute 'vbt'